In [0]:
df = spark.read.csv("/Volumes/workspace/default/batch12-volume/sample data.csv", header=True, inferSchema=True)
display(df)

In [0]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/workspace/default/batch12-volume/Order_Excel.csv")
#display(df) # to display in tabular format
#df.show() #By default, it shows only 20 rows
df.printSchema #to print the schema

In [0]:
df = spark.read.csv("/Volumes/workspace/default/batch12-volume/Order_Excel.csv", header=True, inferSchema=True)
display(df)

#Define schema progarmatically

In [0]:
#Define schema progarmatically
from pyspark.sql.types import *
orderSchema = StructType([StructField("Region", StringType() ,True)
,StructField("Country", StringType() ,True)
,StructField("ItemType", StringType() ,True)
,StructField("SalesChannel", StringType() ,True)
,StructField("OrderPriority", StringType() ,True)
,StructField("OrderID", IntegerType() ,True)
,StructField("UnitsSold", IntegerType() ,True)
,StructField("UnitPrice", DoubleType() ,True)
,StructField("UnitCost", DoubleType() ,True)
,StructField("TotalRevenue", DoubleType() ,True)
,StructField("TotalCost", DoubleType() ,True)
,StructField("TotalProfit", DoubleType() ,True)
])


df = spark.read.load("/Volumes/workspace/default/batch12-volume/Order_Excel.csv",format="csv", header=True, schema=orderSchema)
df.printSchema()
df.schema

# Define schema Declaratively wayb(easy way 😅💪😍)

In [0]:
orderSchema = 'Region String ,Country String ,ItemType String ,SalesChannel String ,OrderPriority String ,OrderID Integer ,UnitsSold Integer ,UnitPrice Double ,UnitCost Double ,TotalRevenue Double ,TotalCost Double ,TotalProfit Double'
                          
df = spark.read.load("/Volumes/workspace/default/batch12-volume/Order_Excel.csv",format="csv", header=True, schema=orderSchema)
df.printSchema()
#df.schema
#display(df)

# HOW to pick particular coulmn using select function 

In [0]:
df1 = df.select("Region","Country","SalesChannel")
display(df1)

In [0]:
#Alternative ways for selecting the particular or specfic column

from pyspark.sql.functions import *
df2 = df.select("Region",col("Country"), column("SalesChannel"), df["SalesChannel"], df.TotalCost)
display(df1)


# Alias and EXPR 

In [0]:
#Alias
from pyspark.sql.functions import *
df3 = df.select("Region",col("Country").alias("New_country"), column("SalesChannel"), df["SalesChannel"], df.TotalCost, expr("TotalProfit*100/TotalCost").alias("profit%"))
display(df1)

In [0]:
df4 = df.withColumn("ProfitPercent", expr("TotalProfit*100/TotalCost"))
display(df4)

# To creat SQLview

In [0]:
df.createOrReplaceTempView("orders")
df5 = spark.sql("select Region, Country from orders where Country ='India' ")
display(df5)

In [0]:
%sql
select * from orders;

## 1) Add new column
## 2) Rename column
## 3) Delete the column

In [0]:
from pyspark.sql.functions import *
df6 = df.withColumn("StoreName", lit("Mumbai-Dmart"))
display(df6)

In [0]:
df7 = df.withColumn("StoreName", lit("Mumbai-Dmart"))
display(df7)
df8 = df.withColumn("ProfitPercent",expr("TotalProfit*100/TotalCost"))
display(df8)
df9 = df.withColumn("ProfitPercent",col("TotalProfit")*100/col("TotalCost"))
display(df9)
###All 3 worrks same

In [0]:
#To rename the col name
df10 =df.withColumnRenamed("Country", "New_Country")
display(df10)

In [0]:
#to delete the col
df11 =df.drop("Country")
display(df11)

## to filter the records we use where or filter

In [0]:
df12 = df.where("Country = 'India'")
display(df12)

df13 = df.filter("Country = 'India' ")
display(df13)

In [0]:
df14 = df.where((col("UnitsSold")>100) & (col("Region") == 'Asia'))
display(df14)
df15 = df.filter((col("UnitsSold")>100) & (col("Region") == 'Asia'))
display(df15)


# Orderby or sortand group by

In [0]:
df16 = df.orderBy(col("Country").desc())
display(df16)
df17 = df.orderBy(col("Country").asc(), col("Region").asc())
display(df17)
df18 = df.sort(col("Country").asc(), col("Region").asc())
display(df18)

#Group by with aggrigate functions

In [0]:
df19 = df.groupBy("Country","Region").agg(max("TotalCost"),min("TotalProfit"),avg("TotalProfit"),sum("TotalRevenue"))
display(df19)

In [0]:
#You have to load a orders data. Now find out the following.
# 1. Display all the country where Totalprofit is more than 10000
# You have to load a orders data. Now find out the following.
# 2. Display all the Orders details where Region is Asia and Country is India
# 3. Diaply all the orders details where UnitsSold is > 100 and result will be 
# displayed in the decreasing order of the UnitsSold
# 4. Display count of orders generated for every SalesChannel
# 5. Display count of orders and sum of the revenue generated for country India and China separately.


In [0]:
df = spark.read.load("/Volumes/workspace/default/batch12-volume/Order_Excel.csv",format="csv", header=True, schema=orderSchema)
# 1. Display all the country where Totalprofit is more than 10000
#df = spark.sql(" select Country from orders where TotalProfit >10000")
# select Country from Order_Excel where Totalprofit > 10000
# You have to load a orders data. Now find out the following.
df20= df.where(col("Totalprofit")> 10000)
df21 = df20.select("Country")
display(df21)

In [0]:
#2) Display all the Orders details where Region is Asia and Country is India
# select * from orders where Region = 'Asia' and Country = 'India'
from pyspark.sql.functions import *
df = spark.read.load("/Volumes/workspace/default/batch12-volume/Order_Excel.csv",format="csv", header=True, schema=orderSchema)
df22 = df.where((col("Region") == 'Asia') & (col("Country") == 'India'))
display(df22)


In [0]:
# 3. Diaply all the orders details where UnitsSold is > 100 and result will be 
# displayed in the decreasing order of the UnitsSold
df23 = df.where(col("UnitsSold") > 100)
df24 =df23.orderBy(col("UnitsSold").desc())
display(df24)

In [0]:
# 4. Display count of orders generated for every SalesChannel
df24 = df.groupBy("SalesChannel").agg(count("*"))
display(df24)

In [0]:
# 5. Display count of orders and sum of the revenue generated for country India and China separately.

df25 = df.where((col("Country")== 'India') | (col("Country")== 'China'))
display(df25)
df26 = df25.groupBy("Country").agg(sum("TotalRevenue").alias("TotalRevenue"), count("*").alias("OrderCount"))
display(df26)

In [0]:
%sql
-- # 5. Display count of orders and sum of the revenue generated for country India and China separately

SELECT Country, COUNT(*) AS order_count, SUM(TotalRevenue) AS total_revenue
FROM Orders
WHERE Country IN ('India', 'China')
GROUP BY Country;



How to play with DATE 

In [0]:
from pyspark.sql.functions import *

df27= spark.read.csv("/Volumes/workspace/default/batch12-volume/dates-batch11.txt", header=True, inferSchema=True)
display(df27)

In [0]:
df28 = df27.withColumn("ConvetToDateType", to_date(col("reg_date"), 'dd-MM-yyyy'))
display(df28)

In [0]:
df29 = df28.withColumn("ConvetTimeStampType", to_timestamp(col("reg_date"), 'dd-MM-yyyy'))
display(df29)

In [0]:
df30 = df29.withColumn("CurentDate", current_date())
df31 = df30.withColumn("CurrentTimeStamp", current_timestamp())
df32 = df31.select(year("CurentDate"),month("CurentDate"),dayofmonth("CurentDate"), hour("CurrentTimeStamp"), minute("CurrentTimeStamp"),second("CurrentTimeStamp"))
display(df32)


In [0]:
df33 = df28.select(date_format("ConvetToDateType", 'dd/MM/yyyy'))
display(df33)